In [22]:
import pandas as pd
import numpy as np
import bs4
import requests
from shapely.geometry import Point
import geopandas as gpd
import warnings
import argparse

def data_adquisition_datosmadrid_json(url):
    response_df = requests.get(url).json()
    response_df.keys()
    res_df = pd.json_normalize(response_df["@graph"])
    return res_df

#main
url1 = "https://datos.madrid.es/egob/catalogo/202318-0-escuelas-infantiles.json"
url2 = "https://datos.madrid.es/egob/catalogo/202311-0-colegios-publicos.json"
url3 = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'
url4 = 'https://openapi.emtmadrid.es/v1/transport/bicimad/stations/' 
warnings.filterwarnings('ignore')


In [23]:
df1 = data_adquisition_datosmadrid_json(url1)
df2 = data_adquisition_datosmadrid_json(url2)

In [24]:
def concatenate_databases(df1,df2):
    conca_df = pd.concat([df1, df2])
    concatenated_df = conca_df.loc[:,['id','title','address.locality','address.postal-code','address.street-address','location.latitude','location.longitude']]
    return concatenated_df

places_of_interest_df = concatenate_databases(df1,df2)

In [25]:
def add_key_column (df):
    df['key'] = int(1)
    return df['key']

In [26]:
add_key_column(places_of_interest_df)

0      1
1      1
2      1
3      1
4      1
      ..
243    1
244    1
245    1
246    1
247    1
Name: key, Length: 309, dtype: int64

In [27]:
places_of_interest_df

,id,title,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,key
0,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1
1,4549323,Escuela infantil municipal Altair,MADRID,28044,CALLE JOSE DE CADALSO 43,40.378132,-3.768701,1
2,5299172,Escuela infantil municipal Amanecer,MADRID,28032,CALLE TITANIO 7,40.401990,-3.601044,1
3,5633312,Escuela infantil municipal Ana de Austria,MADRID,28050,CALLE ANA DE AUSTRIA 32,40.495522,-3.655737,1
4,4724002,Escuela infantil municipal Campanilla,MADRID,28020,CALLE AVILA 4,40.452700,-3.701998,1
...,...,...,...,...,...,...,...,...
243,5326,Colegio Público Virgen del Cortijo,MADRID,28050,AVENIDA MANOTERAS 63,40.487254,-3.660022,1
244,5319,Colegio Público Vázquez de Mella,MADRID,28005,CALLE BAILEN 18,40.412095,-3.713999,1
245,4690940,Colegio Público Winston Churchill,MADRID,28032,AVENIDA GRAN VIA DEL ESTE 5,40.402966,-3.595314,1
246,5096,Colegio Público de Practicas Asunción Rincón,MADRID,28003,AVENIDA FILIPINAS 5,40.439916,-3.710968,1


In [28]:
def data_adquisition_APIBiciMad_json(url1,url2):
    header={'email':'irene9ago85@hotmail.com','password':'Pepper24Merlot.-'}
    get_token=requests.get(url1, headers=header).json()
    accessToken=get_token['data'][0]['accessToken']
    header = {'accessToken':str(accessToken)}
    my_dataset=requests.get(url2, headers = header).json()
    bicimad_original_df = pd.json_normalize(my_dataset['data'])
    df_bicimad = bicimad_original_df.loc[:,['id','name','address','dock_bikes','free_bases','geometry.coordinates']]
    return df_bicimad

In [29]:
imported_bicimad_df = data_adquisition_APIBiciMad_json(url3,url4)

In [30]:
display(imported_bicimad_df)

,id,name,address,dock_bikes,free_bases,geometry.coordinates
0,1,Puerta del Sol A,Puerta del Sol nº 1,16,9,"[-3.7018341, 40.4172137]"
1,2,Puerta del Sol B,Puerta del Sol nº 1,26,4,"[-3.701602938060457, 40.41731271011562]"
2,3,Miguel Moya,Calle Miguel Moya nº 1,16,4,"[-3.7058415, 40.4205886]"
3,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,10,5,"[-3.7069171, 40.4302937]"
4,5,Malasaña,Calle Manuela Malasaña nº 5,19,2,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...
259,265,INEF,Avenida Juan de Herrera frente a la calle Paul...,2,20,"[-3.72997, 40.43896]"
260,266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,0,24,"[-3.72699, 40.44375]"
261,267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,0,24,"[-3.72693, 40.44342]"
262,268,Facultad Biología,Calle José Antonio Novais frente al nº 12,7,17,"[-3.7272945, 40.4483322]"


In [31]:
imported_bicimad_df.keys()

Index(['id', 'name', 'address', 'dock_bikes', 'free_bases',
       'geometry.coordinates'],
      dtype='object')

In [32]:
def split_lat_long_and_cleaning(df_bicimad):
    #funtion to separate latitud and longitude in two columns, to be able to make calculations for the project.
    split_df = pd.DataFrame(df_bicimad['geometry.coordinates'].to_list(), columns = ['location.longitude', 'location.latitude'])
    df_bicimad['geometry.coordinates'] = split_df['location.longitude']
    df_bicimad = df_bicimad.rename(columns={'geometry.coordinates':'location.longitude'})
    df_bicimad['location.latitude'] = split_df['location.latitude']
    #change the object type of columns "latitude" and "longitude" to float
    bicimad_df = df_bicimad.astype({'location.latitude': float, 'location.longitude': float})
    #create a new dataframe with selected columns. 
    return bicimad_df

In [33]:
#display(bicimad_df)

In [34]:
#def split_lat_long_and_cleaning(bicimad_df):
#bicimad_df[['location.longitude','location.latitude']] = bicimad_df['geometry.coordinates'].str.split(',',expand=True)
#bicimad_df['location.longitude'] = bicimad_df['location.longitude'].str.replace("[","")
#bicimad_df['location.latitude'] = bicimad_df['location.latitude'].str.replace("]","")
#bicimad_df = bicimad_df.astype({'location.latitude': float, 'location.longitude': float})
    #new_bicimad_df = bicimad_df.loc[:,['id','name','address','dock_bikes','free_bases','location.longitude','location.latitude']]
    #return bicimad_df

In [35]:
bicimad_cleaned_df = split_lat_long_and_cleaning(imported_bicimad_df)

In [36]:
display(bicimad_cleaned_df)

,id,name,address,dock_bikes,free_bases,location.longitude,location.latitude
0,1,Puerta del Sol A,Puerta del Sol nº 1,16,9,-3.701834,40.417214
1,2,Puerta del Sol B,Puerta del Sol nº 1,26,4,-3.701603,40.417313
2,3,Miguel Moya,Calle Miguel Moya nº 1,16,4,-3.705842,40.420589
3,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,10,5,-3.706917,40.430294
4,5,Malasaña,Calle Manuela Malasaña nº 5,19,2,-3.702587,40.428552
...,...,...,...,...,...,...,...
259,265,INEF,Avenida Juan de Herrera frente a la calle Paul...,2,20,-3.729970,40.438960
260,266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,0,24,-3.726990,40.443750
261,267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,0,24,-3.726930,40.443420
262,268,Facultad Biología,Calle José Antonio Novais frente al nº 12,7,17,-3.727295,40.448332


In [37]:
def filtering_BiciMad_stations_by_bike_availability(df,column_name):
    bicimad_filtered_df = df[df[column_name] > 1]
    return bicimad_filtered_df

In [38]:
bicimad_filtered_df = filtering_BiciMad_stations_by_bike_availability(bicimad_cleaned_df,'dock_bikes')
add_key_column(bicimad_filtered_df)

0      1
1      1
2      1
3      1
4      1
      ..
257    1
258    1
259    1
262    1
263    1
Name: key, Length: 251, dtype: int64

In [39]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [40]:
def apply_mercator_to_df(df):
    df['mercator'] = df.apply(lambda df: to_mercator(df['location.latitude'],df['location.longitude']), axis = 1)
    return df['mercator']

In [41]:
apply_mercator_to_df(bicimad_filtered_df)

0      POINT (4499223.648366921 -412373.2851572837)
1       POINT (4499234.670122574 -412347.498528782)
2      POINT (4499599.340516399 -412820.3206106317)
3      POINT (4500679.707306497 -412940.3068148255)
4      POINT (4500485.866677178 -412457.3286497012)
                           ...                     
257    POINT (4504388.460857612 -410511.0384549898)
258    POINT (4502467.086446521 -413711.4637311997)
259    POINT (4501644.435409558 -415511.9577777162)
262    POINT (4502687.743941171 -415213.4906984274)
263     POINT (4502994.74083288 -415445.0243226733)
Name: mercator, Length: 251, dtype: object

In [42]:
apply_mercator_to_df(places_of_interest_df)

0      POINT (4503325.457708918 -413214.0871607283)
1      POINT (4494873.081456062 -419832.6705785314)
2       POINT (4497528.948937292 -401130.596425248)
3      POINT (4507940.837033544 -407231.1670990717)
4      POINT (4503174.019987035 -412391.5771022496)
                           ...                     
243    POINT (4507020.494757262 -407709.1747154693)
244    POINT (4498653.885099497 -413730.2642768721)
245    POINT (4497637.552307622 -400491.4291664587)
246    POINT (4501750.852518209 -413392.1668429679)
247    POINT (4499705.618399205 -401821.8746131732)
Name: mercator, Length: 309, dtype: object

In [43]:
def merging_by_column (df_1, df_2,column_name):
    merged_df = pd.merge(df_1,df_2, on=column_name, how="outer")
    return merged_df

In [44]:
merged_df = merging_by_column (places_of_interest_df,bicimad_filtered_df,"key")
display(merged_df)

,id_x,title,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,id_y,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y
0,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),1,Puerta del Sol A,Puerta del Sol nº 1,16,9,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837)
1,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),2,Puerta del Sol B,Puerta del Sol nº 1,26,4,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782)
2,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),3,Miguel Moya,Calle Miguel Moya nº 1,16,4,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317)
3,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,10,5,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148255)
4,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),5,Malasaña,Calle Manuela Malasaña nº 5,19,2,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77554,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),263,López Pozas,Calle General López Pozas nº 2,2,21,-3.685140,40.463610,POINT (4504388.460857612 -410511.0384549898)
77555,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),264,Reina Victoria,Calle Beatriz de Bobadilla frente al nº 2,3,21,-3.713830,40.446350,POINT (4502467.086446521 -413711.4637311997)
77556,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),265,INEF,Avenida Juan de Herrera frente a la calle Paul...,2,20,-3.729970,40.438960,POINT (4501644.435409558 -415511.9577777162)
77557,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),268,Facultad Biología,Calle José Antonio Novais frente al nº 12,7,17,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274)


In [45]:
def distance_meters(start,finish):
    # return the distance in metres between to two points
    return start.distance(finish)

def apply_distance_to_df(df):
    df['distance'] = df.apply(lambda df: distance_meters(df['mercator_x'],df['mercator_y']), axis = 1)
    total_sorted_df = df.sort_values(["title", "distance"], ascending = (True, True))
    reindex_total_sorted_df = total_sorted_df.reset_index(drop=True)
    return reindex_total_sorted_df

In [46]:
total_df = apply_distance_to_df(merged_df)
display(total_df)

,id_x,title,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,id_y,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y,distance
0,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),257,Manuel Caldeiro,Paseo de la Castellana nº 298,9,15,-3.685320,40.479380,POINT (4506143.969227422 -410531.1174823334),3796.394240
1,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),260,Chamartín,Calle Rodríguez Jaén nº 40,7,16,-3.683910,40.471600,POINT (4505277.903589051 -410373.8318768776),3976.932693
2,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),225,Pedro Rico,Calle Pedro Rico nº 4,2,22,-3.688463,40.481086,POINT (4506333.880278716 -410881.7202647889),4092.005819
3,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),262,Pío XII,Avenida Pío XII nº 3,4,19,-3.677120,40.460410,POINT (4504032.238487074 -409616.4103570179),4166.338771
4,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),256,Torre Cepsa,Calle Sinesio Delgado nº 2,11,13,-3.687970,40.474350,POINT (4505584.032188732 -410826.7258551974),4257.242824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77554,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),265,INEF,Avenida Juan de Herrera frente a la calle Paul...,2,20,-3.729970,40.438960,POINT (4501644.435409558 -415511.9577777162),8345.739423
77555,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),220,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9,4,19,-3.716591,40.398247,POINT (4497112.284980891 -414019.4641230708),8530.084221
77556,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),231,Ermita Santo,Paseo de la Ermita del Santo nº 48,11,10,-3.724489,40.406838,POINT (4498068.615450309 -414900.5668678451),8759.590980
77557,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),224,Puerta del Ángel,Plaza Puerta del Ángel nº 4,15,8,-3.728318,40.413764,POINT (4498839.629519532 -415327.6677751078),8814.713984


In [47]:
def filtering_closest_bicimad_station_to_place_of_interest(df):
    groupby_df = df.groupby(["title"])["distance"].min()
    total_groupby_df = pd.DataFrame(groupby_df)
    total_groupby_df_windex = total_groupby_df.reset_index(drop = False)
    final_complete_df = total_groupby_df_windex.merge(df,how="left")
    return final_complete_df


In [48]:
final_df_complete_version = filtering_closest_bicimad_station_to_place_of_interest(total_df)
display(final_df_complete_version)

,title,distance,id_x,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,id_y,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y
0,Colegio Público Adolfo Suárez,3796.394240,176850,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),257,Manuel Caldeiro,Paseo de la Castellana nº 298,9,15,-3.685320,40.479380,POINT (4506143.969227422 -410531.1174823334)
1,Colegio Público Agustina Díez,1932.123857,5075,MADRID,28038,CALLE VESUBIO 5,40.388191,-3.647237,1,POINT (4495992.849193973 -406283.0125532473),234,Sierra Toledana,Camino de Valderribas nº 94,7,16,-3.660777,40.399015,POINT (4497197.81342333 -407793.364764792)
2,Colegio Público Agustín Rodriguez Sahagún,3856.153931,4693138,MADRID,28031,CALLE LA GRANJA DE SAN ILDEFONSO 17,40.368040,-3.622168,1,POINT (4493749.655446447 -403486.7215186193),229,Pavones,Calle Hacienda de Pavones nº 342,11,12,-3.634587,40.400368,POINT (4497348.393620864 -404872.003688147)
3,Colegio Público Alberto Alcocer,3253.471117,5076,MADRID,28022,AVENIDA CANILLEJAS A VICALVARO 56,40.433338,-3.612052,1,POINT (4501018.577505093 -402358.4139760713),223,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,13,10,-3.640994,40.429706,POINT (4500614.284841758 -405586.6676448087)
4,Colegio Público Alcalde de Móstoles,6261.761115,5077,MADRID,28044,CALLE ALDEANUEVA DE LA VERA 9,40.379837,-3.776448,1,POINT (4495062.882345807 -420696.9779405064),232,Caramuel,Calle Caramuel nº 51,3,20,-3.728534,40.409136,POINT (4498324.476616229 -415351.7319152079)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Escuela infantil municipal Osa Menor,182.720201,187607,MADRID,28004,CALLE MEJIA LEQUERICA 21,40.428040,-3.699502,1,POINT (4500428.821429083 -412113.1013101297),58,Barceló,Calle Barceló nº 7,19,0,-3.700423,40.426683,POINT (4500277.743757191 -412215.8739185985)
305,Escuela infantil municipal Parque de Ingenieros,3798.993043,11508438,MADRID,28041,CALLE LA TABERNERA DEL PUERTO 26,40.355056,-3.704144,1,POINT (4492304.303298196 -412630.916716107),221,Glorieta de Cádiz,Calle Antonio López nº 129,12,11,-3.700170,40.388950,POINT (4496077.347674986 -412187.6512511889)
306,Escuela infantil municipal Ruiz Jiménez,188.194006,88758,MADRID,28028,CALLE JOSE PICON 17,40.435093,-3.672375,1,POINT (4501213.942059446 -409087.0970815064),199,Diego de león,Calle Alonso Heredia nº 1,5,12,-3.674000,40.434639,POINT (4501163.401625942 -409268.3776613389)
307,Escuela infantil municipal Tarabilla,949.064501,168220,MADRID,28017,CALLE GANDHI 14,40.423692,-3.634964,1,POINT (4499944.775217957 -404914.0010923386),223,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,13,10,-3.640994,40.429706,POINT (4500614.284841758 -405586.6676448087)


In [49]:
def save_data_to_csv (df,path):
    df.to_csv(path, index = False)

In [50]:
#path1 = "../data/example2.csv"
#save_data_to_csv (final_df_complete_version,path1)

In [51]:
#project1_fdf = pd.DataFrame(columns = ["Place of interest","Place address","BiciMAD station","Station location","Available Bikes"])
#project1_fdf[["Place of interest","Place address","BiciMAD station","Station location","Available Bikes"]] = final_df_complete_version[["title","address.street-address","name","address","dock_bikes"]]
#display(project1_fdf)

In [32]:
#print ("This program will return the closest bicimad station to a group of or an specific <place of interest>.")
#print ("First select the output you rather obtain, 2 options are available:")
#print ("by entering the number <1> you will get a list of all <places of interes> and their nearest bicimad station.")
#print ("by entering the number <2> you will get the nearest bicimad station to the specific <place of interest> you enter later on.")
#x = int(input("Please select which output you rather get: 1 or 2."))
#if x = 1 :
#    
#    else:
#    
#
#two options for the final user to select when executing using argparse: (1) To get the table for every 'Place of interest' included in the dataset (or a set of them), (2) To get the table for a specific 'Place of interest' imputed by the user.

In [52]:
def direction_converter_for_google_maps(df,column_to_convert,new_column_name):
    final_df_complete_version[new_column_name] = df[column_to_convert].str.replace(" ","+")
    return final_df_complete_version[new_column_name]

In [53]:
direction_converter_for_google_maps(final_df_complete_version,"title","point_of_interest_google_maps")
direction_converter_for_google_maps(final_df_complete_version,"name","BiciMad_google_maps")
final_df_complete_version["url_google_maps"] = "https://www.google.com/maps/dir/"+final_df_complete_version["point_of_interest_google_maps"]+"/"+"Bicimad"+"+"+final_df_complete_version["BiciMad_google_maps"]
display(final_df_complete_version)

,title,distance,id_x,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,...,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y,point_of_interest_google_maps,BiciMad_google_maps,url_google_maps
0,Colegio Público Adolfo Suárez,3796.394240,176850,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),...,Manuel Caldeiro,Paseo de la Castellana nº 298,9,15,-3.685320,40.479380,POINT (4506143.969227422 -410531.1174823334),Colegio+Público+Adolfo+Suárez,Manuel+Caldeiro,https://www.google.com/maps/dir/Colegio+Públic...
1,Colegio Público Agustina Díez,1932.123857,5075,MADRID,28038,CALLE VESUBIO 5,40.388191,-3.647237,1,POINT (4495992.849193973 -406283.0125532473),...,Sierra Toledana,Camino de Valderribas nº 94,7,16,-3.660777,40.399015,POINT (4497197.81342333 -407793.364764792),Colegio+Público+Agustina+Díez,Sierra+Toledana,https://www.google.com/maps/dir/Colegio+Públic...
2,Colegio Público Agustín Rodriguez Sahagún,3856.153931,4693138,MADRID,28031,CALLE LA GRANJA DE SAN ILDEFONSO 17,40.368040,-3.622168,1,POINT (4493749.655446447 -403486.7215186193),...,Pavones,Calle Hacienda de Pavones nº 342,11,12,-3.634587,40.400368,POINT (4497348.393620864 -404872.003688147),Colegio+Público+Agustín+Rodriguez+Sahagún,Pavones,https://www.google.com/maps/dir/Colegio+Públic...
3,Colegio Público Alberto Alcocer,3253.471117,5076,MADRID,28022,AVENIDA CANILLEJAS A VICALVARO 56,40.433338,-3.612052,1,POINT (4501018.577505093 -402358.4139760713),...,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,13,10,-3.640994,40.429706,POINT (4500614.284841758 -405586.6676448087),Colegio+Público+Alberto+Alcocer,Gutierre+de+Cetina,https://www.google.com/maps/dir/Colegio+Públic...
4,Colegio Público Alcalde de Móstoles,6261.761115,5077,MADRID,28044,CALLE ALDEANUEVA DE LA VERA 9,40.379837,-3.776448,1,POINT (4495062.882345807 -420696.9779405064),...,Caramuel,Calle Caramuel nº 51,3,20,-3.728534,40.409136,POINT (4498324.476616229 -415351.7319152079),Colegio+Público+Alcalde+de+Móstoles,Caramuel,https://www.google.com/maps/dir/Colegio+Públic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Escuela infantil municipal Osa Menor,182.720201,187607,MADRID,28004,CALLE MEJIA LEQUERICA 21,40.428040,-3.699502,1,POINT (4500428.821429083 -412113.1013101297),...,Barceló,Calle Barceló nº 7,19,0,-3.700423,40.426683,POINT (4500277.743757191 -412215.8739185985),Escuela+infantil+municipal+Osa+Menor,Barceló,https://www.google.com/maps/dir/Escuela+infant...
305,Escuela infantil municipal Parque de Ingenieros,3798.993043,11508438,MADRID,28041,CALLE LA TABERNERA DEL PUERTO 26,40.355056,-3.704144,1,POINT (4492304.303298196 -412630.916716107),...,Glorieta de Cádiz,Calle Antonio López nº 129,12,11,-3.700170,40.388950,POINT (4496077.347674986 -412187.6512511889),Escuela+infantil+municipal+Parque+de+Ingenieros,Glorieta+de+Cádiz,https://www.google.com/maps/dir/Escuela+infant...
306,Escuela infantil municipal Ruiz Jiménez,188.194006,88758,MADRID,28028,CALLE JOSE PICON 17,40.435093,-3.672375,1,POINT (4501213.942059446 -409087.0970815064),...,Diego de león,Calle Alonso Heredia nº 1,5,12,-3.674000,40.434639,POINT (4501163.401625942 -409268.3776613389),Escuela+infantil+municipal+Ruiz+Jiménez,Diego+de+león,https://www.google.com/maps/dir/Escuela+infant...
307,Escuela infantil municipal Tarabilla,949.064501,168220,MADRID,28017,CALLE GANDHI 14,40.423692,-3.634964,1,POINT (4499944.775217957 -404914.0010923386),...,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,13,10,-3.640994,40.429706,POINT (4500614.284841758 -405586.6676448087),Escuela+infantil+municipal+Tarabilla,Gutierre+de+Cetina,https://www.google.com/maps/dir/Escuela+infant...


In [54]:
final_df_complete_version["url_google_maps"] = "https://www.google.com/maps/dir/"+final_df_complete_version["point_of_interest_google_maps"]+"/"+"Bicimad"+"+"+final_df_complete_version["BiciMad_google_maps"]

In [55]:
project1_fdf = pd.DataFrame(columns = ["Place of interest","Place address","BiciMAD station","Station location","Distance","Available Bikes","URL google maps"])
project1_fdf[["Place of interest","Place address","BiciMAD station","Station location","Distance","Available Bikes","URL google maps"]] = final_df_complete_version[["title","address.street-address","name","address","distance","dock_bikes","url_google_maps"]]
display(project1_fdf)

,Place of interest,Place address,BiciMAD station,Station location,Distance,Available Bikes,URL google maps
0,Colegio Público Adolfo Suárez,CALLE INFANTA CATALINA MICAELA 31,Manuel Caldeiro,Paseo de la Castellana nº 298,3796.394240,9,https://www.google.com/maps/dir/Colegio+Públic...
1,Colegio Público Agustina Díez,CALLE VESUBIO 5,Sierra Toledana,Camino de Valderribas nº 94,1932.123857,7,https://www.google.com/maps/dir/Colegio+Públic...
2,Colegio Público Agustín Rodriguez Sahagún,CALLE LA GRANJA DE SAN ILDEFONSO 17,Pavones,Calle Hacienda de Pavones nº 342,3856.153931,11,https://www.google.com/maps/dir/Colegio+Públic...
3,Colegio Público Alberto Alcocer,AVENIDA CANILLEJAS A VICALVARO 56,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,3253.471117,13,https://www.google.com/maps/dir/Colegio+Públic...
4,Colegio Público Alcalde de Móstoles,CALLE ALDEANUEVA DE LA VERA 9,Caramuel,Calle Caramuel nº 51,6261.761115,3,https://www.google.com/maps/dir/Colegio+Públic...
...,...,...,...,...,...,...,...
304,Escuela infantil municipal Osa Menor,CALLE MEJIA LEQUERICA 21,Barceló,Calle Barceló nº 7,182.720201,19,https://www.google.com/maps/dir/Escuela+infant...
305,Escuela infantil municipal Parque de Ingenieros,CALLE LA TABERNERA DEL PUERTO 26,Glorieta de Cádiz,Calle Antonio López nº 129,3798.993043,12,https://www.google.com/maps/dir/Escuela+infant...
306,Escuela infantil municipal Ruiz Jiménez,CALLE JOSE PICON 17,Diego de león,Calle Alonso Heredia nº 1,188.194006,5,https://www.google.com/maps/dir/Escuela+infant...
307,Escuela infantil municipal Tarabilla,CALLE GANDHI 14,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,949.064501,13,https://www.google.com/maps/dir/Escuela+infant...


In [56]:
save_data_to_csv (final_df_complete_version,"../data/BiciMad_project_complete_version.csv")

In [30]:
project1_fdf = pd.DataFrame(columns = ["Place of interest","Place address","BiciMAD station","Station location","Distance","Available Bikes","URL google maps"])
project1_fdf[["Place of interest","Place address","BiciMAD station","Station location","Distance","Available Bikes","URL google maps"]] = final_df_complete_version[["title","address.street-address","name","address","distance","dock_bikes","url_google_maps"]]

0      https://www.google.com/maps/dir/Colegio+Públic...
1      https://www.google.com/maps/dir/Colegio+Públic...
2      https://www.google.com/maps/dir/Colegio+Públic...
3      https://www.google.com/maps/dir/Colegio+Públic...
4      https://www.google.com/maps/dir/Colegio+Públic...
                             ...                        
304    https://www.google.com/maps/dir/Escuela+infant...
305    https://www.google.com/maps/dir/Escuela+infant...
306    https://www.google.com/maps/dir/Escuela+infant...
307    https://www.google.com/maps/dir/Escuela+infant...
308    https://www.google.com/maps/dir/Escuela+infant...
Name: url_google_maps, Length: 309, dtype: object

In [ ]:
print(final_df_complete_version["url_google_maps"])

In [58]:
x = str(input("Please enter the education center from which you would like to calculate the closest Bicimad station: "))

Please enter the education center from which you would like to calculate the closest Bicimad station: Colegio Público Adolfo Suárez


In [61]:
print ("The closest Bicimad station to", x,"is: ")
print (project1_fdf.loc[project1_fdf['Place of interest'] == x])

The closest Bicimad station to Colegio Público Adolfo Suárez is: 
               Place of interest                      Place address  \
0  Colegio Público Adolfo Suárez  CALLE INFANTA CATALINA MICAELA 31   

   BiciMAD station               Station location    Distance  \
0  Manuel Caldeiro  Paseo de la Castellana nº 298  3796.39424   

   Available Bikes                                    URL google maps  
0                9  https://www.google.com/maps/dir/Colegio+Públic...  


In [57]:
googlemaps_url = (final_df_complete_version.loc[final_df_complete_version['title'] == x]['url_google_maps'].values[0])
webbrowser(googlemaps_url)

NameError: name 'x' is not defined

In [ ]:
get(https://openapi.emtmadrid.es/v2/mobilitylabs/user/login/)

In [ ]:
https://mynavega.emtmadrid.es/MapViewer/?state={%22visibleLayers%22%3A{%22Movilidad_8158%22%3A[]}%2C%22center%22%3A%22-412623%2C4926545%2C102100%22%2C%22level%22%3A6}

In [46]:
response = requests.get('https://openapi.emtmadrid.es/v2/mobilitylabs/user/login/', {
  'headers': {
    'user': 'irene9ago85@hotmail',
    'password': "Pepper24Merlot.-"
  }
});

In [47]:
print (response.text)

{"code": "90", "description": "Error in mobilitylabs_context Init session", "datetime": "2022-01-19T21:32:06.130438", "data": []}


In [48]:
#Se puede utilizar a partir de los 14 años. Los menores de 16 deberán ser dados de alta por un adulto, en calidad de tutor o representante legal, y asumirá completamente la responsabilidad tanto de los actos de los menores durante la utilización del servicio como de su idoneidad física para usar el servicio.
#El casco es obligatorio para los menores de 16 años.

#https://www.bicimad.com/index.php?s=preguntas
#https://www.madrid.es/portales/munimadrid/es/Inicio/Movilidad-y-transportes/Oficina-de-la-bici/Preguntas-frecuentes-sobre-la-bicicleta-en-la-ciudad?vgnextoid=d7a45b1f61fc8210VgnVCM2000000c205a0aRCRD&vgnextchannel=125331dc4f768210VgnVCM2000000c205a0aRCRD

#print ("This program will return the closest bicimad station to a group of or an specific <place of interest>.")
#print ("First select the output you rather obtain, 2 options are available:")
#print ("by entering the number <1> you will get a list of all <places of interes> and their nearest bicimad station.")
#print ("by entering the number <2> you will get the nearest bicimad station to the specific <place of interest> you enter later on.")
#x = int(input("Please select which output you rather get: 1 or 2."))
#if x = 1 :
#    
#    else:
#    
#
#two options for the final user to select when executing using argparse: (1) To get the table for every 'Place of interest' included in the dataset (or a set of them), (2) To get the table for a specific 'Place of interest' imputed by the user.

In [52]:
parser = argparse.ArgumentParser()
parser.parse_args()

parser = argparse.ArgumentParser(description="This program wil return the closest bicimad station to a group of or an specific <place of interest>.")
#parser.add_argument("-g", "--group_by_category", action="store_true", choices = ["Escuelas infantiles - Preschool","Colegios Públicos - Public Schools"],
#                    help ="returns the Bicimad stations nearest to an specific group of places of interest)
parser.add_argument("-l", "--list",
                    help ="displays a list of all places of interest available in the data.")
parser.add.argument("-s", "--single_option",
                    help = "displays the nearest BiciMad station with availability to a single place of interest.")
parser.add.argument("-a", "--all",
                    help = "displays a list of all <places of interes> and their nearest bicimad station."
parser.add_argument("-i", "--information",
                    help ="displays the <frequent questions> page of Bicimad.")
parser.add_argument("-m", "BiciMad stations map",
                    help ="displays a map of all Bicimad stations.")
                    
if args.list:
    print("The list of the places of interest:}")
    print(final_df_complete_version[title])
elif args.single_option:
    x = str(input("Please enter the education center from which you would like to calculate the closest Bicimad station: "))
    print ("The closest Bicimad station to", x,"is: ")
    print (project1_fdf.loc[project1_fdf['Place of interest'] == x])
    print (final_df_complete_version.loc[final_df_complete_version['title'] == x]['url_google_maps'].values[0])
elif args.all:
    print (project1_fdf)
    


NameError: name 'argparse' is not defined

{'code': '01',
 'description': 'Token e5a5a4cf-2990-44ce-bad9-c8944ea38222 extend  into control-cache Data recovered  OK, (lapsed: 34 millsecs)',
 'datetime': '2022-01-20T19:32:10.120738',
 'data': [{'nameApp': 'OPENAPI MobilityLabs',
   'levelApp': 0,
   'updatedAt': '2022-01-18T20:16:12.7170000',
   'userName': 'irene9',
   'lastUpdate': {'$date': 1642692588596},
   'idUser': 'e7d24afc-d1f0-491b-96c6-ea513b7e4e80',
   'priv': 'U',
   'tokenSecExpiration': 86399,
   'email': 'irene9ago85@hotmail.com',
   'tokenDteExpiration': {'$date': 1642782587596},
   'flagAdvise': True,
   'accessToken': 'e5a5a4cf-2990-44ce-bad9-c8944ea38222',
   'apiCounter': {'current': 1,
    'dailyUse': 20000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!',
    'aboutUses': 'If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in htt

In [71]:
get_token.keys()

dict_keys(['code', 'description', 'datetime', 'data'])

In [76]:
accessToken=get_token['data'][0]['accessToken']

In [77]:
accessToken

'e5a5a4cf-2990-44ce-bad9-c8944ea38222'

In [100]:
url='https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'
header = {'accessToken':str(accessToken)}
my_dataset=requests.get(url, headers = header).json()
bicimad_original_df = pd.json_normalize(my_dataset['data'])
bicimad_df = bicimad_original_df.loc[:,['id','name','address','dock_bikes','free_bases','geometry.coordinates']]
display (bicimad_df)

,id,name,address,dock_bikes,free_bases,geometry.coordinates
0,1,Puerta del Sol A,Puerta del Sol nº 1,17,9,"[-3.7018341, 40.4172137]"
1,2,Puerta del Sol B,Puerta del Sol nº 1,21,8,"[-3.701602938060457, 40.41731271011562]"
2,3,Miguel Moya,Calle Miguel Moya nº 1,9,12,"[-3.7058415, 40.4205886]"
3,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,7,8,"[-3.7069171, 40.4302937]"
4,5,Malasaña,Calle Manuela Malasaña nº 5,16,7,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...
259,265,INEF,Avenida Juan de Herrera frente a la calle Paul...,3,19,"[-3.72997, 40.43896]"
260,266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,0,24,"[-3.72699, 40.44375]"
261,267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,11,13,"[-3.72693, 40.44342]"
262,268,Facultad Biología,Calle José Antonio Novais frente al nº 12,3,20,"[-3.7272945, 40.4483322]"


In [80]:

    return res_df

[{'id': 1,
  'name': 'Puerta del Sol A',
  'light': 2,
  'number': '1a',
  'address': 'Puerta del Sol nº 1',
  'activate': 1,
  'no_available': 0,
  'total_bases': 30,
  'dock_bikes': 17,
  'free_bases': 9,
  'reservations_count': 0,
  'geometry': {'type': 'Point', 'coordinates': [-3.7018341, 40.4172137]}},
 {'id': 2,
  'name': 'Puerta del Sol B',
  'light': 1,
  'number': '1b',
  'address': 'Puerta del Sol nº 1',
  'activate': 1,
  'no_available': 0,
  'total_bases': 30,
  'dock_bikes': 21,
  'free_bases': 8,
  'reservations_count': 0,
  'geometry': {'type': 'Point',
   'coordinates': [-3.701602938060457, 40.41731271011562]}},
 {'id': 3,
  'name': 'Miguel Moya',
  'light': 2,
  'number': '2',
  'address': 'Calle Miguel Moya nº 1',
  'activate': 1,
  'no_available': 0,
  'total_bases': 24,
  'dock_bikes': 9,
  'free_bases': 12,
  'reservations_count': 1,
  'geometry': {'type': 'Point', 'coordinates': [-3.7058415, 40.4205886]}},
 {'id': 4,
  'name': 'Plaza Conde Suchil',
  'light': 2,
 

In [99]:
final_df_complete_version.loc[final_df_complete_version['title'] == x]['url_google_maps'].values[0]

'https://www.google.com/maps/dir/Colegio+Público+Agustín+Rodriguez+Sahagún/Bicimad+Pavones'